In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import CXGate
# from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler import CouplingMap
from qiskit.circuit.library import C4XGate
from qiskit_aer import AerSimulator
# from qiskit.providers.fake_provider import FakeCambridge
QiskitRuntimeService.delete_account()

import time
import os
# QiskitRuntimeService.save_account(token="IYxMCq8Fhs5NzG80K3br2EhvbU5WziWlfypyuyr3SELB",channel="ibm_quantum",overwrite=True )
# Set the API key as an environment variable

# Initialize the Qiskit Runtime Service
your_api_key = "V_5F6RZPUb9GB37hSH3CQrQpdFX7WpfP2kek91zjo0Ml"
your_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/5e194ca79679454da57a2ad2d3e49a9d:749f65fa-d77a-44f6-913a-5a77858ae7aa::"


QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=your_api_key,
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/5e194ca79679454da57a2ad2d3e49a9d:749f65fa-d77a-44f6-913a-5a77858ae7aa::",
    overwrite=True
)
service = QiskitRuntimeService()
# Specify the backend
backend_name = 'ibm_brisbane'
backend = service.backend(backend_name)

sim =AerSimulator()
from collections import Counter, defaultdict
from qiskit.circuit.random import random_circuit
from qiskit import transpile
import numpy as np
np.random.seed(42)
from qiskit.quantum_info import state_fidelity, partial_trace, DensityMatrix
# from qiskit_aer.primitives import Sampler  # Aer Sampler runs locally
from qiskit_ibm_runtime import Sampler
from qiskit import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
 
# Get a fake backend from the fake provider
backend = FakeManilaV2()
# Assume backend is defined somewhere earlier in your environment
sampler = Sampler(backend)  # no backend needed, runs on AerSimulator under the hood
from collections import Counter
import numpy as np
from qiskit.circuit.library import SwapGate, CXGate, RZZGate, RXXGate, RYYGate, iSwapGate
from qiskit.quantum_info import Operator
def is_swap_gate(instruction_tuple):
    """Check if the given instruction represents a SWAP gate or equivalent sequence."""
    instruction = instruction_tuple[0]  # Unpack the instruction from the tuple

    # Direct SWAP gate
    if isinstance(instruction, SwapGate):
        return True
    
    # Named SWAP gate
    if instruction.name.lower() == 'swap':
        return True
    
    # Check for equivalent operations using matrix comparison
    if instruction.num_qubits == 2:
        swap_op = Operator(SwapGate())
        instr_op = Operator(instruction)
        if np.allclose(instr_op.data, swap_op.data, atol=1e-10):
            return True
    
    # Check for common SWAP implementations
    if isinstance(instruction, CXGate):
        # CNOT-based SWAP (3 CNOTs)
        if hasattr(instruction, 'definition') and instruction.definition:
            data = instruction.definition.data
            if len(data) == 3:
                cx1, cx2, cx3 = data[0][0], data[1][0], data[2][0]
                if (isinstance(cx1, CXGate) and isinstance(cx2, CXGate) and isinstance(cx3, CXGate) and
                    cx1.qargs[0] == cx2.qargs[1] and cx1.qargs[1] == cx3.qargs[1] and
                    cx2.qargs[0] == cx3.qargs[0]):
                    return True
    
    # iSWAP-based SWAP
    if isinstance(instruction, iSwapGate):
        if hasattr(instruction, 'definition') and instruction.definition:
            data = instruction.definition.data
            if len(data) == 2:
                iswap, s = data[0][0], data[1][0]
                if isinstance(iswap, iSwapGate) and s.name == 's' and s.qargs == iswap.qargs:
                    return True
    
    # RZZ, RXX, RYY-based SWAP
    if isinstance(instruction, (RZZGate, RXXGate, RYYGate)):
        if hasattr(instruction, 'definition') and instruction.definition:
            data = instruction.definition.data
            if len(data) == 3:
                rzz, rxx, ryy = data[0][0], data[1][0], data[2][0]
                if (isinstance(rzz, RZZGate) and isinstance(rxx, RXXGate) and isinstance(ryy, RYYGate) and
                    rzz.params[0] == rxx.params[0] == ryy.params[0] == np.pi/2 and
                    rzz.qargs == rxx.qargs == ryy.qargs):
                    return True
    
    return False

from qiskit import QuantumCircuit
from qiskit.circuit.library.standard_gates import SwapGate
# Updated function to derive coupling map
# Updated function to derive coupling map
# Updated function to derive coupling map
def derive_coupling_map_from_circuit(transpiled_circuit):
    derived_coupling_map = set()  # Use a set to avoid duplicate entries

    # Initialize the swap history for each qubit in the circuit
    qubit_swap_history = {qubit: False for qubit in transpiled_circuit.qubits}

    for instruction in transpiled_circuit.data:
        if instruction[0].name == 'swap':
            # If the instruction is a SWAP gate, mark the qubits as involved in a swap
            for qarg in instruction[1]:
                qubit_swap_history[qarg] = True
            continue  # Move to the next instruction

        if len(instruction[1]) == 2:
            # Directly use the qubit objects
            qubits = [qarg for qarg in instruction[1]]

            # Check if any of the qubits were involved in a SWAP
            if not any(qubit_swap_history[qubit] for qubit in qubits):
                # If neither qubit has been swapped, add the connection
                qubit_indices = [transpiled_circuit.find_bit(qarg).index for qarg in qubits]
                derived_coupling_map.add(tuple(sorted(qubit_indices)))

    return derived_coupling_map
def standardize_counts(counts, num_qubits):
    """Convert all keys to little-endian bitstring format used by Qiskit get_counts()."""
    new_counts = {}
    for k, v in counts.items():
        if isinstance(k, int):  # integer key
            bitstring = format(k, f"0{num_qubits}b")[::-1]  # reverse for little-endian
        elif isinstance(k, (list, tuple)):  # tuple or list of bits
            bitstring = "".join(str(b) for b in reversed(k))  # reverse order
        else:  # assume string
            bitstring = k
        new_counts[bitstring] = new_counts.get(bitstring, 0) + v
    return new_counts

def get_ideal_counts(circuit, shots=1000):
    # Ensure circuit has measurements for counts

    
    job = sim.run(circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts
def normalize_counts(counts):
    """Normalize counts dictionary to probabilities."""
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}

def classical_fidelity(p, q):
    """Calculate classical fidelity (Bhattacharyya coefficient) between two distributions p and q."""
    all_keys = set(p.keys()).union(q.keys())
    fidelity = 0
    for k in all_keys:
        fidelity += np.sqrt(p.get(k, 0) * q.get(k, 0))
    return fidelity

def compute_circuit_fidelity(qc, backend, shots=1000):
    cr = ClassicalRegister(1, name='c')
    qc.add_register(cr)


    # Allowed gates (no "dcx")
    allowed_gates = ["cx", "h", "x", "y", "z", "rx", "ry", "rz", "sx", "u", "id"]

  

    # Convert to only allowed gates (transpile)

    qc = transpile(qc, basis_gates=allowed_gates)

    # Measure qubit 1 into the classical bit 0 of 'c'
    qc.measure(2, cr[0])
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    qc_isa = pm.run(qc)

    # -------------------
    # 1) Run on real backend (or Sampler)
    sampler = Sampler(mode=backend)
    counts_backend = sampler.run([qc_isa], shots=1000).result()[0].data.c.get_counts()
    print("Counts from backend:", counts_backend)
    backend = AerSimulator()
    # -------------------
    # 2) Run same transpiled circuit on statevector simulator for fidelity
    pm = generate_preset_pass_manager(backend=sim, optimization_level=3)
    qc_isa = pm.run(qc)

    # -------------------
    # 1) Run on real backend (or Sampler)
    sampler = Sampler(mode=backend)
    counts_sim = sampler.run([qc_isa], shots=1000).result()[0].data.c.get_counts()
    print("Counts from backend:", counts_sim)

    all_keys = set(counts_backend.keys()).union(set(counts_sim.keys()))
    fidelity = sum(
        (counts_backend.get(k, 0) / shots) ** 0.5 *
        (counts_sim.get(k, 0) / shots) ** 0.5
        for k in all_keys
    ) ** 2

    print("Classical fidelity:", fidelity)
    return fidelity


def compute_transpiler_fidelity_expression(transpiled_circuit, derived_coupling_map, backend):
    """
    Build a symbolic fidelity LHS expression from the transpiled circuit,
    ignoring virtual-Z type gates (rz, u1, u2).
    """
    basis_gates = backend.configuration().basis_gates  # get native gate set

    transpiled_circuit = transpile(
        transpiled_circuit,
        backend=backend,
        basis_gates=basis_gates,
        optimization_level=3
    )

    gate_counts = defaultdict(int)
    readout_qubits = set()

    # gates to ignore (virtual Z rotations)
    ignore_gates = {"rz", "u1", "u2", "p"}

    for inst, qargs, _ in transpiled_circuit.data:
        if not qargs:
            continue  # skip barriers, etc.
        q_indices = tuple(transpiled_circuit.find_bit(qarg).index for qarg in qargs)
        sorted_indices = tuple(sorted(q_indices))

        if inst.name in ignore_gates:
            continue  # skip virtual-Z gates

        # Only include 2Q gates present in derived coupling map
        if len(q_indices) == 2 and sorted_indices not in derived_coupling_map:
            continue

        if inst.name == "measure":
            readout_qubits.add(q_indices[0])
        else:
            gate_counts[(inst.name, sorted_indices)] += 1

    # 🔹 Build symbolic expression
    symbolic_terms = []
    for (gname, qubits), count in gate_counts.items():
        symbolic_terms.append(f"(1 - ε_{gname}{qubits})^{count}")

    for q in sorted(readout_qubits):
        symbolic_terms.append(f"(1 - ε_ro_q{q})")

    return " * ".join(symbolic_terms) if symbolic_terms else "1"

def build_fidelity_equation_with_rhs(transpiled_circuit, derived_coupling_map, shots=1000):
    lhs_expr = compute_transpiler_fidelity_expression(transpiled_circuit, derived_coupling_map, backend)
    rhs_fid = compute_circuit_fidelity(transpiled_circuit, backend, shots)
    rhs_value = f"{rhs_fid:.6f}"
    return f"{lhs_expr} = {rhs_value}"

from qiskit.circuit.random import random_circuit
import numpy as np

equation_list = []


for i in range(16):  # more circuits
    num_qubits = 5
    # depth = np.random.randint(5, 30)  # deeper circuits
    qc = random_circuit(num_qubits, 2, max_operands=2, seed=42+i)
    big_rc = derive_coupling_map_from_circuit(qc)
    symbolic = build_fidelity_equation_with_rhs(qc, big_rc, shots=1000)
    equation_list.append(symbolic)



print("\n".join(equation_list))


In [ ]:
# ----------------- comparison function -----------------
def compare_error_values(all_solutions, backend, tol=1e-6, plot=True, compute_accuracy=True):
    """
    Compare one or more solutions (list of dicts) against backend error values.
    Returns a list of tuples:
      (matches, mismatches, missing, accuracies, single_acc, two_acc, stats)
    """
    def get_backend_val(gate, qubits):
        # Special handling: map 'ecr' to 'cx' if backend doesn't have ecr
        try:
            return backend.properties().gate_error(gate, qubits)
        except:
            if gate == "ecr":
                try:
                    return backend.properties().gate_error("cx", qubits)
                except:
                    return None
            return None

    results = []

    if isinstance(all_solutions, dict):  # wrap single dict in list
        all_solutions = [all_solutions]

    for sol in all_solutions:
        matches, mismatches, missing = {}, {}, {}
        accuracies = {}
        single_qubit_acc = {}
        two_qubit_acc = {}

        for key, val in sol.items():
            clean_key = key[4:] if key.startswith("eps_") else key
            parts = clean_key.split("_")
            gate = parts[0]
            qubits = tuple(map(int, parts[1:])) if len(parts) > 1 else ()

            backend_val = get_backend_val(gate, qubits)

            if backend_val is None:
                # excluded from all accuracy/statistics calculations
                missing[key] = val
                continue  

            # if backend value exists → compare
            if abs(val - backend_val) <= tol:
                matches[key] = (val, backend_val)
            else:
                mismatches[key] = (val, backend_val)

            if compute_accuracy:
                acc = 1 - abs(val - backend_val) / (backend_val if backend_val != 0 else 1e-12)
                acc = max(0.0, min(1.0, acc))
                accuracies[key] = acc

                if len(qubits) == 1:
                    single_qubit_acc[key] = acc
                elif len(qubits) == 2:
                    two_qubit_acc[key] = acc

        # --- compute stats ---
        stats = {}
        for label, group in [("single", single_qubit_acc), ("two", two_qubit_acc)]:
            if group:
                vals = np.array(list(group.values()))
                stats[f"{label}_mean"] = float(np.mean(vals))
                stats[f"{label}_min"] = float(np.min(vals))
                stats[f"{label}_max"] = float(np.max(vals))
            else:
                stats[f"{label}_mean"] = None
                stats[f"{label}_min"] = None
                stats[f"{label}_max"] = None

        results.append((matches, mismatches, missing, accuracies, single_qubit_acc, two_qubit_acc, stats))

    if plot:
        plot_error_comparison(results)

    return results


# ----------------- plotting function -----------------
def plot_error_comparison(results):
    """
    Plot solver vs backend error rates for all solutions (log scale).
    Missing backend values are marked in orange.
    """
    for i, (matches, mismatches, missing, accuracies, single_qubit_acc, two_qubit_acc, stats) in enumerate(results):
        fig, ax = plt.subplots(figsize=(10, 6))

        keys = []
        solver_vals = []
        backend_vals = []
        missing_keys = []

        # matches + mismatches
        combined = {**matches, **mismatches}
        for k, (s, b) in combined.items():
            keys.append(k)
            solver_vals.append(s)
            backend_vals.append(b)

        # missing backend points
        for k, s in missing.items():
            keys.append(k)
            solver_vals.append(s)
            backend_vals.append(None)
            missing_keys.append(len(keys)-1)

        x = np.arange(len(keys))

        # Solver points
        ax.scatter(x, solver_vals, color='blue', marker='x', s=80, label='Solver')

        # Backend points where available
        for xi, b in enumerate(backend_vals):
            if b is not None:
                ax.scatter(xi, b, color='red', marker='o', s=80, label='Backend' if xi==0 else "")
                ax.plot([xi, xi], [solver_vals[xi], b], color='gray', linestyle='--', alpha=0.6)

        # Highlight missing backend points
        # if missing_keys:
        #     ax.scatter([x[k] for k in missing_keys],
        #                [solver_vals[k] for k in missing_keys],
        #                color='orange', marker='o', s=80, label='Missing backend')

        ax.set_yscale("log")
        ax.set_ylabel("Error rate (log scale)",fontsize=20)
        ax.set_xticks(x)
        ax.set_xticklabels(keys, rotation=90,fontsize=16)
        ax.tick_params(axis='y', labelsize=16)                  # bigger ytick labels
        ax.legend(fontsize=16) 
        plt.tight_layout()
        plt.show()

# sol is your single solution dict from least_squares
comparison_results = compare_error_values(sol, backend, tol=1e-6)

matches, mismatches, missing, accuracies, single_acc, two_acc, stats = comparison_results[0]

print("Single-qubit accuracies:", single_acc)
print("Two-qubit accuracies:", two_acc)
print("Stats:", stats)
